In [1]:
%matplotlib inline
import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import warnings
warnings.filterwarnings('ignore')

In [2]:
from quat_math import quat2AxisAngle

def quats2Point(quats):
    pts = []
    for q in quats:
        xi, theta = quat2AxisAngle(q)
        pts.append(xi*theta)
    return np.array(pts)

def scatterSO3(vertices, vals, q_gt = None, alpha_max = 1.0):
    #c = cmap(vals)
    a = np.maximum(0,np.minimum(1, (vals - min(vals))/(max(vals) - min(vals))))
    c = cmap(a)
    
    c[:,3] = alpha_max*a

    pts = quats2Point(vertices)
    ax.scatter(pts[:,0], pts[:,1], pts[:,2], s=a*10, c=c)

    if(q_gt is not None):
        pt_gt = quats2Point(q_gt)
        ax.scatter(pt_gt[:,0], pt_gt[:,1], pt_gt[:,2], c='r', marker='x')

In [3]:
from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset

from object_pose_utils.datasets.image_processing import ImageNormalizer
from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes
from object_pose_utils.datasets.inplane_rotation_augmentation import InplaneRotator

dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'
object_list = list(range(1,22))
mode = "test"

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.IMAGE_CROPPED, 
                 otypes.DEPTH_POINTS_MASKED_AND_INDEXES]



In [4]:
from dense_fusion.network import PoseNetDropout
model_checkpoint = '/home/bokorn/src/DenseFusion/trained_models/ycb_dropout/pose_model_17_0.023200422725836868.pth'

estimator = PoseNetDropout(num_points = 1000, num_obj = 21)
estimator.load_state_dict(torch.load(model_checkpoint, map_location=lambda storage, loc: storage))
estimator.cuda();

In [5]:
from dense_fusion.network import PoseNet

estimator_mean = PoseNet(num_points = 1000, num_obj = 21)
estimator_mean.load_state_dict(torch.load(model_checkpoint, map_location=lambda storage, loc: storage))
estimator_mean.cuda();
estimator_mean.eval()

PoseNet(
  (cnn): ModifiedResnet(
    (model): DataParallel(
      (module): PSPNet(
        (feats): ResNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (relu): ReLU(inplace)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): BasicBlock(
              (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (relu): ReLU(inplace)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            )
            (1): BasicBlock(
              (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (relu): ReLU(inplace)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            )
          )
          (layer2): Sequential(
            (0): BasicBlock(

In [7]:
from se3_distributions.datasets.ycb_dataset import getYCBSymmeties
from object_pose_utils.utils.pose_processing import symmetricAngularDistance, meanShift
import pathlib


import scipy.io as scio
from tqdm import tqdm_notebook as tqdm

num_samples = 30

with torch.no_grad():
    for object_id in tqdm(object_list):
        sym_axis, sym_ang = getYCBSymmeties(object_id)
        ycb_dataset = YCBDataset(dataset_root, mode=mode, 
                                 object_list = [object_id], 
                                 output_data = output_format, 
                                 preprocessors = [InplaneRotator(np.pi/2)],
                                 postprocessors = [ImageNormalizer()],
                                 resample_on_error = False,
                                 image_size = [640, 480], num_points=1000)

        pathlib.Path('results/data'.format(ycb_dataset.classes[object_id])).mkdir(parents=True, exist_ok=True)
        
        for j, data in tqdm(enumerate(ycb_dataset), total=len(ycb_dataset)):
            obj, quat, img, points, choose, = data
            if(len(obj) == 0):
                continue
            
            idx = obj - 1
            points, choose, img, idx = Variable(points.unsqueeze(0)).cuda(), \
                                       Variable(choose.unsqueeze(0)).cuda(), \
                                       Variable(img.unsqueeze(0)).cuda(), \
                                       Variable(idx.unsqueeze(0)).cuda()
            
            pred_qs = []
            ap_x, emb, bs = estimator.feature(img, points, choose)
            for _ in range(num_samples):
                pred_r, pred_t, pred_c = estimator.evaluate(ap_x, idx, bs)
                pred_q = pred_r[0,torch.argmax(pred_c)][[1,2,3,0]]
                pred_q /= pred_q.norm()
                pred_qs.append(to_np(pred_q))
            pred_qs = np.stack(pred_qs)

            mean_r, pred_t, pred_c, emb = estimator_mean(img, points, choose, idx)
            mean_q = mean_r[0,torch.argmax(pred_c)][[1,2,3,0]]
            mean_q /= mean_q.norm()
            
            savepath = 'results/{}'.format(ycb_dataset.getPath(j))
            pathlib.Path(savepath).mkdir(parents=True, exist_ok=True)
            
            scio.savemat('{}/{}_dropout_samples_ROT.mat'.format(savepath, 
                         ycb_dataset.classes[object_id]),
                        {'index':j,
                         'quat':to_np(quat),
                         'pred_qs':pred_qs,
                         'mean_q':to_np(mean_q),
                         })

Exception on index 49: Mask 0050/000633 has less than minimum number of pixels (49 < 50)
Exception on index 50: Mask 0050/000640 has less than minimum number of pixels (41 < 50)
Exception on index 51: Mask 0050/000642 has less than minimum number of pixels (45 < 50)
Exception on index 52: Mask 0050/000644 has less than minimum number of pixels (18 < 50)
Exception on index 54: Mask 0050/000653 has less than minimum number of pixels (31 < 50)
Exception on index 55: Mask 0050/000660 has less than minimum number of pixels (16 < 50)
Exception on index 57: Mask 0050/000671 has less than minimum number of pixels (31 < 50)
Exception on index 58: Mask 0050/000681 has less than minimum number of pixels (7 < 50)
Exception on index 59: Mask 0050/000682 has less than minimum number of pixels (2 < 50)
Exception on index 60: Mask 0050/000701 has less than minimum number of pixels (41 < 50)
Exception on index 553: No points in mask
Exception on index 559: No points in mask
Exception on index 560: No p

Exception on index 87: Mask 0049/000833 has less than minimum number of pixels (47 < 50)
Exception on index 108: Mask 0049/001157 has less than minimum number of pixels (2 < 50)
Exception on index 111: Mask 0049/001179 has less than minimum number of pixels (47 < 50)
Exception on index 115: Mask 0049/001222 has less than minimum number of pixels (13 < 50)
Exception on index 116: Mask 0049/001229 has less than minimum number of pixels (10 < 50)
Exception on index 118: Mask 0049/001247 has less than minimum number of pixels (50 < 50)
Exception on index 129: Mask 0049/001340 has less than minimum number of pixels (40 < 50)
Exception on index 138: Mask 0049/001441 has less than minimum number of pixels (25 < 50)
Exception on index 322: No points in mask
Exception on index 323: No points in mask
Exception on index 324: No points in mask
Exception on index 325: No points in mask
Exception on index 326: No points in mask
Exception on index 327: No points in mask
Exception on index 334: No poi

Exception on index 659: No points in mask
Exception on index 661: No points in mask
Exception on index 665: No points in mask
Exception on index 666: No points in mask
Exception on index 671: No points in mask
Exception on index 672: No points in mask
Exception on index 673: No points in mask
Exception on index 674: No points in mask


Exception on index 174: No points in mask
Exception on index 175: No points in mask
Exception on index 176: No points in mask
Exception on index 177: No points in mask
Exception on index 178: No points in mask
Exception on index 179: No points in mask
Exception on index 180: No points in mask


In [10]:
ls results/0048/000001/

002_master_chef_can_dropout_samples.mat
002_master_chef_can_fc6_reg_results.mat
002_master_chef_can_fc6_results.mat
002_master_chef_can_feat_global_reg_results.mat
002_master_chef_can_feat_global_results.mat
002_master_chef_can_feat_global_sigma_results.mat
007_tuna_fish_can_dropout_samples.mat
007_tuna_fish_can_fc6_reg_results.mat
007_tuna_fish_can_fc6_results.mat
007_tuna_fish_can_feat_global_reg_results.mat
007_tuna_fish_can_feat_global_results.mat
007_tuna_fish_can_feat_global_sigma_results.mat
025_mug_dropout_samples.mat
025_mug_fc6_reg_results.mat
025_mug_fc6_results.mat
025_mug_feat_global_reg_results.mat
025_mug_feat_global_results.mat
025_mug_feat_global_sigma_results.mat
051_large_clamp_dropout_samples.mat
051_large_clamp_fc6_reg_results.mat
051_large_clamp_fc6_results.mat
051_large_clamp_feat_global_reg_results.mat
051_large_clamp_feat_global_results.mat
051_large_clamp_feat_global_sigma_results.mat
052_extra_large_clamp_dropout_samples.mat
052_extra_large_clamp_fc6_reg_resu

In [7]:
estimator_dropout = estimator_dropout.eval();

NameError: name 'estimator_dropout' is not defined

In [ ]:

pred_r, pred_t, pred_c, emb = estimator(img, points, choose, idx)
pred_q = pred_r[0,torch.argmax(pred_c)][[1,2,3,0]]
pred_q /= pred_q.norm()

symmetricAngularDistance(pred_q.unsqueeze(0), quat.unsqueeze(0).cuda(),
                         sym_axis, sym_ang).item()*180/np.pi

In [ ]:
estimator_mean.eval()
pred_r, pred_t, pred_c, emb = estimator_mean(img, points, choose, idx)
pred_q = pred_r[0,torch.argmax(pred_c)][[1,2,3,0]]
pred_q /= pred_q.norm()

symmetricAngularDistance(pred_q.unsqueeze(0), quat.unsqueeze(0).cuda(),
                         sym_axis, sym_ang).item()*180/np.pi